In [1]:
#Importing necessaray libraries

import numpy as np
import pandas as pd
import sys
sys.path.append("..")
import os
import matplotlib.pyplot as plt
import tensorflow as tf

from numpy import asarray
from numpy import savez_compressed
from numpy import asarray
from numpy import save
from numpy import asarray
from numpy import savez_compressed
from sklearn import preprocessing

from sklearn.compose import make_column_transformer
from sklearn.preprocessing import OneHotEncoder 
ohe = OneHotEncoder(sparse=False)

from gan import output
sys.modules["output"] = output
from gan.doppelganger import DoppelGANger
from gan.load_data import load_data
from gan.network import DoppelGANgerGenerator, Discriminator, AttrDiscriminator
from gan.output import Output, OutputType, Normalization
import tensorflow as tf
from gan.network import DoppelGANgerGenerator, Discriminator, \
    RNNInitialStateType, AttrDiscriminator
from gan.util import add_gen_flag, normalize_per_sample, \
    renormalize_per_sample



Bad key "text.kerning_factor" on line 4 in
C:\Users\Navid\anaconda3\envs\myenv\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test_patch.mplstyle.
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.1.2/matplotlibrc.template
or from the matplotlib source distribution
C:\Users\Navid\anaconda3\envs\myenv\lib\site-packages\tensorflow\python\framework\dtypes.py:469: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\Navid\anaconda3\envs\myenv\lib\site-packages\tensorflow\python\framework\dtypes.py:470: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\Navid\anaconda3\envs\myenv\lib\site-pack

In [2]:
#creating output arrays for the features

data_feature_outputs = [
output.Output(type_=OutputType.CONTINUOUS,dim=1,normalization=Normalization.ZERO_ONE,is_gen_flag=False), 
# time intervals between transactions (Dif)
    output.Output(type_=OutputType.DISCRETE,dim=20,is_gen_flag=False), 
# binarized Amount
    output.Output(type_=OutputType.DISCRETE,dim=5,is_gen_flag=False),
# Flag
output.Output(type_=OutputType.DISCRETE,dim=44,is_gen_flag=False) 
# Description    
]

In [3]:
#creating output arrays for the attributes
data_attribute_outputs = [
    output.Output(type_=OutputType.CONTINUOUS,dim=0,normalization=Normalization.MINUSONE_ONE,is_gen_flag=False)]

In [4]:
#reading 20k rows from the dataset
row_num = 20000
data=pd.read_csv('data.csv',nrows=row_num)

In [5]:
data.loc[data.loc[((data["Amount"] > -2172) & (data["Amount"] < -1671))].index, 'Amount_bin'] = 0
data.loc[data.loc[((data["Amount"] > -1671) & (data["Amount"] < -1171))].index, 'Amount_bin'] = 1
data.loc[data.loc[((data["Amount"] > -1171) & (data["Amount"] < -670))].index, 'Amount_bin'] = 2
data.loc[data.loc[((data["Amount"] > -670) & (data["Amount"] < -171))].index, 'Amount_bin'] = 3
data.loc[data.loc[((data["Amount"] > -171) & (data["Amount"] < 329))].index, 'Amount_bin'] = 4
data.loc[data.loc[((data["Amount"] > 329) & (data["Amount"] < 829))].index, 'Amount_bin'] = 5
data.loc[data.loc[((data["Amount"] > 829) & (data["Amount"] < 1329))].index, 'Amount_bin'] = 6
data.loc[data.loc[((data["Amount"] > 1329) & (data["Amount"] < 1529))].index, 'Amount_bin'] = 7
data.loc[data.loc[((data["Amount"] > 1529) & (data["Amount"] < 1729))].index, 'Amount_bin'] = 8
data.loc[data.loc[((data["Amount"] > 1729) & (data["Amount"] < 1929))].index, 'Amount_bin'] = 9
data.loc[data.loc[((data["Amount"] > 1929) & (data["Amount"] < 2130))].index, 'Amount_bin'] = 10
data.loc[data.loc[((data["Amount"] > 2130) & (data["Amount"] < 2329))].index, 'Amount_bin'] = 11
data.loc[data.loc[((data["Amount"] > 2329) & (data["Amount"] < 2529))].index, 'Amount_bin'] = 12
data.loc[data.loc[((data["Amount"] > 2529) & (data["Amount"] < 3029))].index, 'Amount_bin'] = 13
data.loc[data.loc[((data["Amount"] > 3029) & (data["Amount"] < 3529))].index, 'Amount_bin'] = 14
data.loc[data.loc[((data["Amount"] > 3529) & (data["Amount"] < 4029))].index, 'Amount_bin'] = 15
data.loc[data.loc[((data["Amount"] > 4029) & (data["Amount"] < 4529))].index, 'Amount_bin'] = 16
data.loc[data.loc[((data["Amount"] > 4529) & (data["Amount"] < 5029))].index, 'Amount_bin'] = 17
data.loc[data.loc[((data["Amount"] > 5029) & (data["Amount"] < 6746))].index, 'Amount_bin'] = 18
data['Amount'] = data['Amount'].fillna(19)
data['Amount'] = data['Amount_bin']
data = data.drop('Amount_bin', axis = 'columns')

In [6]:
#creating arrays for categorical & continuous features
categorical = ['Amount','Flag','Description'] 
continuous =['Balance','Dif']

#concatenating the arrays
cols = categorical + continuous

In [7]:
#number of unique categories in each categorical feature
categories_n = []

In [8]:
#filling number of distinct categories in the categories_n array
for cat in categorical:
    
    #store the number of distinct categories in each category 
    ohe.fit_transform(data[[cat]])
    ohe_Cat = ohe.categories_
    
    #function to concatenate the number into the categories_n array
    def get_all_elements_in_cat(list):
        count = 0
        for element in ohe_Cat:
            count += len(element)
        return count
    categories_n.append(get_all_elements_in_cat(ohe_Cat))


print(categories_n)

[20, 5, 44]


In [9]:
#processor function to OneHotEncode the features
def processor(user):
    
    #OneHotEncode the features 
    column_transform = make_column_transformer(
    (OneHotEncoder(), ['Flag','Description']), 
      remainder = 'passthrough')
    OHencode=column_transform.fit_transform(user)
    
    #put the values in a dataframe
    new_user = pd.DataFrame(OHencode)
    
    #rename the columns back to their original names
    col = (len(new_user.columns))-1
    col_1 = (len(new_user.columns))-2
    col_2 = (len(new_user.columns))-3
    new_user.rename(columns = {col_1:'Balance'}, inplace = True)
    new_user.rename(columns = {col:'Dif'}, inplace = True)
    new_user.rename(columns = {col_2:'Amount'}, inplace = True)
    new_user['Dif'] = new_user['Dif'].fillna(0)
    
    return new_user

In [10]:
def format_data(data, cols, nsequences=200, Lmax=100, cardinality=70):
    
    #unique account ids
    idd=list(data['Account_id'].unique()) 
    
    #format date
    data.Date = pd.to_datetime(data.Date)
    

    #appending Dif to the dataset and adding random numbers to it
    data['Dif']=np.random.randint(0,30*24*3600, size=row_num)
    
    #fill in the Numpy arrays with zeros with the needed sizes
    data_all = np.zeros((nsequences,Lmax,cardinality))
    data_attribut=np.zeros((nsequences))
    data_gen_flag=np.zeros((nsequences,Lmax))
    
    
    for i,ids in enumerate(idd[:nsequences]):
        
        #make a new dataset equivalent to 'data' with unique account ids
        user = data[data.Account_id==ids]
        user=user.sort_values(by='Date')
        
        #determine row differences and fill in the NaN values
        user['Dif']=user.Date.diff(1).iloc[1:] 
        user['Dif']=user['Dif'].dt.seconds
        user['Dif'] = user['Dif'].fillna(user['Dif'].mean())
        user['Dif'] = user['Dif'].fillna(0)
        
        #drop out the unnecessary columns
        user = user[cols]
        
        
        # copy dataset to preprocess
        processed_data = processor(user)

        #processed_data.to_csv('raw_data.csv', index=False)
         
        print(processed_data)
        #fill Data_attribut with the values of normalized 'Balance
        data_attribut[i] = processed_data['Balance'].values[0]
        
        
        #fill data_gen_flag with zeros & ones 
        data_gen_flag[i,:len(user)]=1
        
        processed_array = np.asarray(processed_data.iloc[:])
        #print(processed_data.shape[1])
        user_col = processed_data.shape[1]
        data_all[i,:len(user),:user_col]=processed_array

    return data_all, data_attribut, data_gen_flag

In [11]:
#calling the function to produce the input
data_all, data_attribut, data_gen_flag = format_data(data,cols)

     0    1    2    3    4  Amount  Balance           Dif
0  0.0  1.0  0.0  0.0  1.0     4.0   349.00  42819.333333
1  1.0  0.0  0.0  1.0  0.0     4.0   280.52  48943.000000
2  1.0  0.0  1.0  0.0  0.0     4.0   247.38  78655.000000
3  1.0  0.0  0.0  1.0  0.0     4.0   230.17    860.000000
     0    1    2    3    4  Amount  Balance      Dif
0  0.0  1.0  0.0  0.0  1.0     4.0  1012.75  19800.0
1  1.0  0.0  1.0  0.0  0.0     4.0  1003.76  36000.0
2  0.0  1.0  0.0  1.0  0.0     4.0   982.76   3600.0
     0    1    2    3    4    5    6    7  Amount  Balance       Dif
0  0.0  0.0  1.0  0.0  0.0  0.0  1.0  0.0     4.0  2693.00  32733.75
1  0.0  0.0  1.0  0.0  0.0  1.0  0.0  0.0     0.0   943.00  25200.00
2  0.0  0.0  1.0  0.0  0.0  0.0  0.0  1.0     4.0   921.00      0.00
3  0.0  1.0  0.0  1.0  0.0  0.0  0.0  0.0     4.0   913.01  21600.00
4  1.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0     4.0   797.38  84135.00
     0    1    2    3    4    5    6    7    8    9   10   11  Amount  \
0  0.0  0.0

     0    1    2  Amount  Balance      Dif
0  1.0  0.0  1.0     4.0   549.66  18000.0
1  1.0  1.0  0.0     4.0   517.66  18000.0
     0    1    2    3    4    5    6  Amount  Balance      Dif
0  0.0  1.0  0.0  0.0  0.0  0.0  1.0     4.0   898.00  48600.0
1  1.0  0.0  0.0  0.0  1.0  0.0  0.0     4.0   893.51  28800.0
2  0.0  1.0  0.0  0.0  0.0  1.0  0.0     2.0  -202.49  64800.0
3  1.0  0.0  1.0  0.0  0.0  0.0  0.0     4.0  -210.48  39600.0
4  0.0  1.0  0.0  1.0  0.0  0.0  0.0     4.0  -242.48  61200.0
     0    1    2    3    4    5    6  Amount  Balance       Dif
0  0.0  1.0  1.0  0.0  0.0  0.0  0.0     4.0   451.00  27304.75
1  0.0  1.0  0.0  0.0  0.0  1.0  0.0     2.0  -534.00  21600.00
2  0.0  1.0  0.0  1.0  0.0  0.0  0.0     4.0  -570.42  10800.00
3  0.0  1.0  0.0  0.0  0.0  0.0  1.0     4.0  -600.25   3600.00
4  1.0  0.0  0.0  0.0  1.0  0.0  0.0     8.0  1121.02  73219.00
     0    1    2    3  Amount  Balance      Dif
0  0.0  1.0  0.0  1.0     2.0  -538.00  33566.0
1  1.0  0.0  

5  0.0  1.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0     4.0   177.07  79200.0
     0    1    2    3  Amount  Balance     Dif
0  1.0  0.0  1.0  0.0     4.0   987.01  7200.0
1  0.0  1.0  0.0  1.0     2.0  -160.99  7200.0
     0    1    2    3    4    5    6  Amount  Balance      Dif
0  0.0  1.0  0.0  0.0  0.0  1.0  0.0     4.0   -36.00  27900.0
1  0.0  1.0  0.0  0.0  0.0  0.0  1.0     4.0   -59.00  82800.0
2  0.0  1.0  1.0  0.0  0.0  0.0  0.0     4.0  -100.82      0.0
3  0.0  1.0  0.0  0.0  1.0  0.0  0.0     2.0 -1210.82  10800.0
4  1.0  0.0  0.0  1.0  0.0  0.0  0.0     4.0 -1230.82  18000.0
     0    1    2    3    4    5    6    7  Amount  Balance      Dif
0  0.0  0.0  1.0  0.0  0.0  0.0  1.0  0.0     4.0   -22.00  28800.0
1  0.0  1.0  0.0  1.0  0.0  0.0  0.0  0.0     4.0   -29.99   7200.0
2  0.0  0.0  1.0  0.0  0.0  1.0  0.0  0.0     4.0   -74.80  75600.0
3  1.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0     4.0   -74.80  21600.0
4  0.0  0.0  1.0  0.0  0.0  0.0  0.0  1.0     4.0  -111.47  10800.0


     0    1    2    3    4    5  Amount  Balance      Dif
0  1.0  0.0  1.0  0.0  0.0  0.0     4.0   655.01  38400.0
1  0.0  1.0  0.0  0.0  0.0  1.0     4.0   632.01  28800.0
2  0.0  1.0  0.0  1.0  0.0  0.0     4.0   588.45  14400.0
3  1.0  0.0  0.0  0.0  1.0  0.0     4.0   583.45  72000.0
     0    1    2    3    4    5    6  Amount  Balance      Dif
0  0.0  1.0  0.0  0.0  1.0  0.0  0.0     4.0   301.51  31776.0
1  0.0  0.0  1.0  0.0  0.0  0.0  1.0     4.0   279.51  32400.0
2  1.0  0.0  0.0  0.0  0.0  1.0  0.0     4.0   263.52  11554.0
3  1.0  0.0  0.0  0.0  0.0  1.0  0.0     4.0   254.65  80726.0
4  1.0  0.0  0.0  1.0  0.0  0.0  0.0     4.0   213.08   2424.0
     0    1    2    3    4    5    6    7    8    9  Amount  Balance      Dif
0  0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0     4.0   221.00  34800.0
1  0.0  1.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0     4.0   201.00  36000.0
2  0.0  0.0  1.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0     4.0   155.92   3600.0
3  0.0  0.0  1.0  0.

     0    1    2    3    4    5  Amount  Balance      Dif
0  0.0  1.0  0.0  1.0  0.0  0.0     4.0   -48.41  40379.0
1  0.0  1.0  1.0  0.0  0.0  0.0     4.0   -80.41  10800.0
2  0.0  1.0  0.0  0.0  0.0  1.0     2.0  -958.41  25200.0
3  1.0  0.0  0.0  0.0  1.0  0.0     4.0  -988.89  85137.0
     0    1    2    3    4  Amount  Balance      Dif
0  1.0  0.0  1.0  0.0  0.0     4.0   169.25  48600.0
1  0.0  1.0  0.0  1.0  0.0     4.0   124.36  28800.0
2  0.0  1.0  0.0  0.0  1.0     4.0   102.36  68400.0
     0    1    2    3    4  Amount  Balance      Dif
0  1.0  0.0  0.0  1.0  0.0     4.0   101.05  50400.0
1  0.0  1.0  1.0  0.0  0.0     4.0    68.05  43200.0
2  0.0  1.0  0.0  0.0  1.0     2.0 -1043.95  57600.0
     0    1    2    3    4    5    6    7    8  Amount  Balance      Dif
0  0.0  0.0  1.0  0.0  0.0  0.0  0.0  1.0  0.0     4.0   374.00  36157.8
1  0.0  0.0  1.0  0.0  1.0  0.0  0.0  0.0  0.0     4.0   324.87  10800.0
2  0.0  1.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0     4.0   316.88  10

     0    1    2    3    4    5    6  Amount  Balance      Dif
0  0.0  1.0  0.0  1.0  0.0  0.0  0.0     4.0   441.01  39600.0
1  0.0  0.0  1.0  0.0  0.0  1.0  0.0     2.0  -449.99  79200.0
2  0.0  0.0  1.0  0.0  0.0  0.0  1.0     4.0  -481.99  32400.0
3  1.0  0.0  0.0  0.0  1.0  0.0  0.0     4.0  -564.65   7200.0
     0    1    2    3    4    5  Amount  Balance           Dif
0  0.0  1.0  0.0  0.0  1.0  0.0     4.0   793.00  44340.333333
1  0.0  1.0  0.0  0.0  0.0  1.0     4.0   760.17  28800.000000
2  1.0  0.0  1.0  0.0  0.0  0.0     3.0   296.04  83966.000000
3  1.0  0.0  0.0  1.0  0.0  0.0     4.0   256.02  20255.000000
     0    1    2    3    4    5    6  Amount  Balance      Dif
0  0.0  1.0  0.0  0.0  0.0  1.0  0.0     4.0   107.00  28800.0
1  0.0  1.0  0.0  0.0  1.0  0.0  0.0     4.0    71.00  43200.0
2  0.0  1.0  0.0  1.0  0.0  0.0  0.0     1.0 -1186.00  50400.0
3  0.0  1.0  0.0  0.0  0.0  0.0  1.0     4.0 -1215.83   3600.0
4  1.0  0.0  1.0  0.0  0.0  0.0  0.0     4.0 -1220.32  

     0    1    2    3    4    5  Amount  Balance           Dif
0  0.0  1.0  0.0  0.0  0.0  1.0     4.0   371.75  36388.666667
1  0.0  1.0  1.0  0.0  0.0  0.0     4.0   319.28   7200.000000
2  0.0  1.0  0.0  1.0  0.0  0.0     4.0   286.28  21600.000000
3  1.0  0.0  0.0  0.0  1.0  0.0     4.0   261.29  80366.000000
     0    1    2    3    4    5    6  Amount  Balance       Dif
0  0.0  1.0  0.0  0.0  1.0  0.0  0.0     4.0   -33.00  50399.75
1  0.0  1.0  0.0  0.0  0.0  0.0  1.0     4.0   -65.83  28800.00
2  1.0  0.0  0.0  0.0  0.0  1.0  0.0     3.0  -421.46  80366.00
3  1.0  0.0  1.0  0.0  0.0  0.0  0.0     4.0  -434.62  20311.00
4  0.0  1.0  0.0  1.0  0.0  0.0  0.0     4.0  -481.51  72122.00
     0    1    2    3  Amount  Balance      Dif
0  1.0  1.0  0.0  0.0     4.0   743.81   9000.0
1  1.0  0.0  1.0  0.0     4.0   699.81  18000.0
2  1.0  0.0  0.0  1.0     4.0   669.98      0.0
     0    1    2    3    4    5    6  Amount  Balance      Dif
0  0.0  1.0  0.0  0.0  0.0  0.0  1.0     4.0  

     0    1    2    3    4    5    6  Amount  Balance      Dif
0  0.0  1.0  0.0  1.0  0.0  0.0  0.0     1.0  -556.00  45900.0
1  0.0  1.0  1.0  0.0  0.0  0.0  0.0     4.0  -604.59  75600.0
2  0.0  1.0  0.0  0.0  1.0  0.0  0.0     4.0  -637.59   3600.0
3  1.0  0.0  0.0  0.0  0.0  1.0  0.0     4.0  -662.59  25200.0
4  0.0  1.0  0.0  0.0  0.0  0.0  1.0     4.0  -696.84  79200.0
     0    1    2    3    4    5    6  Amount  Balance           Dif
0  0.0  1.0  0.0  1.0  0.0  0.0  0.0     4.0   456.01  29745.666667
1  0.0  0.0  1.0  0.0  0.0  1.0  0.0     4.0   435.01  79200.000000
2  0.0  0.0  1.0  0.0  0.0  0.0  1.0     4.0   400.76   3600.000000
3  1.0  0.0  0.0  0.0  1.0  0.0  0.0    15.0  4129.68   6437.000000
     0    1    2    3    4    5    6    7  Amount  Balance      Dif
0  0.0  1.0  0.0  1.0  0.0  0.0  0.0  0.0     4.0   144.16  28076.0
1  0.0  1.0  0.0  0.0  0.0  0.0  1.0  0.0     1.0 -1228.84  75600.0
2  0.0  1.0  0.0  0.0  1.0  0.0  0.0  0.0     4.0 -1261.84  10800.0
3  0.0  1.

In [12]:
#setting some variables
sample_len = 100
epoch = 100
batch_size = 20
d_rounds = 2
g_rounds = 1
d_gp_coe = 10.0
attr_d_gp_coe = 10.0
g_attr_d_coe = 1.0

In [13]:
# normalize data
(data_feature, data_attribute, data_attribute_outputs,
 real_attribute_mask) = normalize_per_sample(
        data_all, data_attribut, data_feature_outputs,
        data_attribute_outputs)

# add generation flag to features
data_feature, data_feature_outputs = add_gen_flag(
    data_feature, data_gen_flag, data_feature_outputs, sample_len)
generator = DoppelGANgerGenerator(
    feed_back=False,
    noise=True,
    feature_outputs=data_feature_outputs,
    attribute_outputs=data_attribute_outputs,
    real_attribute_mask=real_attribute_mask,
    sample_len=sample_len,
    feature_num_units=100,
    feature_num_layers=2)
#call the discriminator functions
discriminator = Discriminator()
attr_discriminator = AttrDiscriminator()

In [14]:
#define where the training data are stored
checkpoint_dir = "./test/checkpoint"
sample_dir = "./test/sample"
time_path = "./test/time.txt"

# set some values
epoch = 100
batch_size = 50
g_lr = 0.0001
d_lr = 0.0001
vis_freq = 50
vis_num_sample = 5
d_rounds = 2
g_rounds = 1
d_gp_coe = 10.0
attr_d_gp_coe = 10.0
g_attr_d_coe = 1.0
extra_checkpoint_freq = 30
num_packing = 1

In [15]:
run_config = tf.ConfigProto()
total_generate_num_sample = 100
with tf.Session(config=run_config) as sess:
    gan = DoppelGANger(
        sess=sess, 
        checkpoint_dir=checkpoint_dir,
        sample_dir=sample_dir,
        time_path=time_path,
        epoch=epoch,
        batch_size=batch_size,
        data_feature=data_feature,
        data_attribute=data_attribute,
        real_attribute_mask=real_attribute_mask,
        data_gen_flag=data_gen_flag,
        sample_len=sample_len,
        data_feature_outputs=data_feature_outputs,
        data_attribute_outputs=data_attribute_outputs,
        vis_freq=vis_freq,
        vis_num_sample=vis_num_sample,
        generator=generator,
        discriminator=discriminator,
        attr_discriminator=attr_discriminator,
        d_gp_coe=d_gp_coe,
        attr_d_gp_coe=attr_d_gp_coe,
        g_attr_d_coe=g_attr_d_coe,
        d_rounds=d_rounds,
        g_rounds=g_rounds,
        num_packing=num_packing,
        extra_checkpoint_freq=extra_checkpoint_freq)

# build the network 
    gan.build()
    length = int(data_feature.shape[1] / sample_len)
    real_attribute_input_noise = gan.gen_attribute_input_noise(
                total_generate_num_sample)
    addi_attribute_input_noise = gan.gen_attribute_input_noise(
                total_generate_num_sample)
    feature_input_noise = gan.gen_feature_input_noise(
                total_generate_num_sample, length)
    input_data = gan.gen_feature_input_data_free(
                total_generate_num_sample)
    print('Build completed!')
    
# load the weights / change the path accordingly
    gan.load(checkpoint_dir+'/epoch_id-89')
    print('Weights loaded!')
    
# generate features, attributes and lengths
    features, attributes, gen_flags, lengths = gan.sample_from(
        real_attribute_input_noise, addi_attribute_input_noise,
        feature_input_noise, input_data, given_attribute=None,
        return_gen_flag_feature=False)
    print('Generation completed!')
    
#denormalise accordingly
    features, attributes = renormalize_per_sample(
        features, attributes, data_feature_outputs,
        data_attribute_outputs, gen_flags,
        num_real_attribute=1)
    print('Denormalization completed!')

Layers of DoppelGANgerGenerator
[<tf.Variable 'DoppelGANgerGenerator/attribute_real/layer0/linear/dense/kernel:0' shape=(5, 100) dtype=float32_ref>, <tf.Variable 'DoppelGANgerGenerator/attribute_real/layer0/linear/dense/bias:0' shape=(100,) dtype=float32_ref>, <tf.Variable 'DoppelGANgerGenerator/attribute_real/layer0/batch_norm/beta:0' shape=(100,) dtype=float32_ref>, <tf.Variable 'DoppelGANgerGenerator/attribute_real/layer0/batch_norm/gamma:0' shape=(100,) dtype=float32_ref>, <tf.Variable 'DoppelGANgerGenerator/attribute_real/layer0/batch_norm/moving_mean:0' shape=(100,) dtype=float32_ref>, <tf.Variable 'DoppelGANgerGenerator/attribute_real/layer0/batch_norm/moving_variance:0' shape=(100,) dtype=float32_ref>, <tf.Variable 'DoppelGANgerGenerator/attribute_real/layer1/linear/dense/kernel:0' shape=(100, 100) dtype=float32_ref>, <tf.Variable 'DoppelGANgerGenerator/attribute_real/layer1/linear/dense/bias:0' shape=(100,) dtype=float32_ref>, <tf.Variable 'DoppelGANgerGenerator/attribute_real

Build completed!
INFO:tensorflow:Restoring parameters from ./test/checkpoint/epoch_id-89\model-359
Weights loaded!
Generation completed!
Denormalization completed!


In [16]:
#stacking the attribute values to get the original shape
attributes = np.stack(attributes, axis=1)
Balance = data[cols]
for i in range(attributes.shape[0]):
    for j in range(attributes.shape[1]):
        if (i ==0):
            #multiply  the generated values to shift the floating points
            Balance.iloc[j,3] = attributes[i,j]*10000
        else:
            #multiply  the generated values to shift the floating points
            Balance.iloc[j+attributes.shape[1]] = attributes[i,j]*10000
#drop out extra column to only view 'Balance'            
Balance = Balance.drop(columns= ['Amount','Flag','Description', 'Dif'])
Balance.iloc[:400,:]

C:\Users\Navid\anaconda3\envs\myenv\lib\site-packages\pandas\core\indexing.py:1763: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


,Balance
0,9853.829145
1,4.523424
2,8848.771453
3,8953.568339
4,8914.003968
...,...
395,-42.000000
396,981.750000
397,1701.530000
398,3921.170000


In [17]:
#put the features values of each sequence in a data frame to return the sequences
for i in range(features.shape[0]):
    df = pd.DataFrame()
    for j in range(8):
        ser = pd.Series(features[i,j,:8])
        df = df.append(ser, ignore_index = True)
    print(df)  

          0         1         2         3         4         5         6  \
0  0.951992  0.045188  0.038076  0.037671  0.037399  0.043326  0.106203   
1  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
2  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
3  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
4  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
5  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
6  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
7  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   

          7  
0  0.042169  
1  0.000000  
2  0.000000  
3  0.000000  
4  0.000000  
5  0.000000  
6  0.000000  
7  0.000000  
          0         1         2         3         4         5         6  \
0 -0.000647  0.040755  0.038578  0.026098  0.021762  0.072945  0.059119   
1 -0.000000  0.000000  0.000000  0.000000  0.000

          0         1         2         3         4         5         6  \
0  1.236153  0.032272  0.060696  0.046451  0.046566  0.028872  0.046883   
1  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
2  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
3  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
4  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
5  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
6  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
7  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   

          7  
0  0.048466  
1  0.000000  
2  0.000000  
3  0.000000  
4  0.000000  
5  0.000000  
6  0.000000  
7  0.000000  
          0         1         2         3         4         5         6  \
0 -0.033302  0.065424  0.082619  0.058176  0.023569  0.017602  0.029419   
1 -0.075081  0.030135  0.035071  0.036386  0.055

          0         1         2         3         4         5         6  \
0  0.828596  0.050397  0.035883  0.054688  0.054336  0.032848  0.033722   
1  0.919799  0.064759  0.032733  0.072448  0.059264  0.035065  0.050597   
2  0.872033  0.048387  0.040942  0.040094  0.067045  0.064553  0.048850   
3  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
4  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
5  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
6  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
7  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   

          7  
0  0.029061  
1  0.062206  
2  0.034663  
3  0.000000  
4  0.000000  
5  0.000000  
6  0.000000  
7  0.000000  
          0         1         2         3         4         5         6  \
0 -0.023074  0.047948  0.043916  0.044598  0.069160  0.100694  0.096613   
1  0.015585  0.039939  0.029983  0.068711  0.029

          0         1         2         3         4         5         6  \
0  0.028068  0.031881  0.039345  0.019375  0.028219  0.026071  0.049306   
1 -0.016135  0.039260  0.035817  0.026541  0.019726  0.045955  0.060910   
2  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
3  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
4  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
5 -0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
6 -0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
7  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   

          7  
0  0.028931  
1  0.036435  
2  0.000000  
3  0.000000  
4  0.000000  
5  0.000000  
6  0.000000  
7  0.000000  
         0        1         2         3         4         5         6  \
0  0.86261  0.04017  0.027682  0.034343  0.067852  0.058444  0.028158   
1  0.00000  0.00000  0.000000  0.000000  0.000000  0

          0         1         2         3         4         5         6  \
0  1.024971  0.016169  0.030069  0.025631  0.023653  0.044334  0.037173   
1  0.985048  0.042668  0.035643  0.019139  0.047387  0.126766  0.153458   
2  0.660206  0.027053  0.055949  0.031319  0.032207  0.049112  0.024498   
3  1.090788  0.058455  0.029958  0.019721  0.043948  0.035294  0.036103   
4  1.151484  0.038105  0.046844  0.042676  0.031776  0.050705  0.049649   
5  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
6  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
7  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   

          7  
0  0.055666  
1  0.059111  
2  0.054868  
3  0.035247  
4  0.037251  
5  0.000000  
6  0.000000  
7  0.000000  
          0         1         2         3         4         5         6  \
0  1.023051  0.050088  0.019183  0.027796  0.021736  0.039000  0.038614   
1  0.551472  0.034673  0.012434  0.037830  0.031

          0         1         2         3         4         5         6  \
0  0.797004  0.022130  0.046255  0.065274  0.019856  0.034391  0.042365   
1  0.985811  0.055099  0.021433  0.022231  0.031489  0.161093  0.045401   
2  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
3  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
4  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
5  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
6  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
7  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   

          7  
0  0.046078  
1  0.040410  
2  0.000000  
3  0.000000  
4  0.000000  
5  0.000000  
6  0.000000  
7  0.000000  
          0        1         2        3         4         5         6  \
0 -0.059053  0.02328  0.027799  0.03272  0.039824  0.054728  0.035629   
1  0.000000  0.00000  0.000000  0.00000  0.000000  0